In [1]:
using CSV, DataFrames, JuMP, Gurobi, StatsPlots, Random, Statistics

## Load in data

In [2]:
# Define sizes
#num_suppliers = 200 # i
#num_consumer_regions = 7 # j = c(i) 
#num_time = 10; # t 
#num_producer_regions = 5 # p(i)

# Parameters
# alpha
# delta

In [3]:
availability = CSV.read("availability.csv", DataFrame)
col_order = [1, 9, 8, 7, 6, 5, 4, 3, 2]
availability = availability[:, col_order]
availability = sort(availability, :Region)

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,AMERICAS,4898050,5500841,5313444,5454975,6324931,6109370,7043821,4628549
2,EMEA,727388,816895,789212,810092,939168,907185,1045899,687513
3,N ASIA,28864415,32415483,31312111,32145270,37273118,36002052,41507653,27275215
4,S ASIA,76623904,86051270,83122231,85334026,98946691,95572216,110187650,72405283
5,SE ASIA,100819493,113223358,109369617,112279721,130190750,125750659,144981558,95268541


In [4]:
demand = CSV.read("demand.csv", DataFrame)
replace!(demand.Region, "Europe, Middle East, and Africa" => "EMEA")
demand

Row,Region,2016,2017,2018,2019,2020,2021,2022,2023
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Asia Pacific,1.84351e7,2.06687e7,2.24933e7,2.27891e7,2.17006e7,2.30219e7,2.58658e7,2.85839e7
2,EMEA,4.32876e7,4.45665e7,5.04292e7,5.40172e7,5.05751e7,5.98283e7,6.34163e7,7.09013e7
3,Greater China,2.2309e7,2.50644e7,3.00086e7,3.65837e7,3.97854e7,4.93391e7,4.64893e7,4.66524e7
4,Latin America,6.71511e6,7.52871e6,8.19335e6,8.30106e6,7.90458e6,8.38586e6,9.42178e6,1.04119e7
5,North America,7.98197e7,8.31245e7,8.00172e7,8.62232e7,8.00773e7,9.99485e7,1.04961e8,1.27871e8


In [5]:
revenue = CSV.read("rev.csv", DataFrame)
replace!(revenue.Region, "Europe, Middle East, and Africa" => "EMEA")
rev_year = []
for i in 2:size(revenue)[2]
    col_sum = sum(revenue[:, i]) * 1e6
    append!(rev_year, col_sum)
end
rev_year = Array(rev_year);

In [6]:
holding_costs = CSV.read("inventory_holding_cost.csv", DataFrame)[:, 2:end]

Row,Year,holding_cost_per_shoe
,Int64,Float64
1,2016,43.92
2,2017,36.75
3,2018,30.22
4,2019,37.57
5,2020,43.73
6,2021,35.65
7,2022,36.63
8,2023,38.44


In [7]:
shipping_costs = CSV.read("transport_costs.csv", DataFrame);

In [8]:
shipping_mapping = shipping_costs[:, 1:2]
shipping_mapping = sort(shipping_mapping, [:producer_region, :consumer_region], rev=[false, false])
shipping_mapping = hcat(DataFrame(Row_Count=1:nrow(shipping_mapping)), shipping_mapping)
shipping_mapping = combine(groupby(shipping_mapping, [:producer_region, :consumer_region])) do sub_df
    DataFrame(Value_mean = first(sub_df.Row_Count))
end
shipping_mapping = unstack(shipping_mapping, :consumer_region, :Value_mean)
shipping_mapping = sort(shipping_mapping, :producer_region)

Row,producer_region,Asia Pacific,Greater China,EMEA,North America,Latin America
,String15,Int64?,Int64?,Int64?,Int64?,Int64?
1,AMERICAS,1,3,2,5,4
2,EMEA,6,8,7,10,9
3,N ASIA,11,13,12,15,14
4,S ASIA,16,18,17,20,19
5,SE ASIA,21,23,22,25,24


In [9]:
production_costs = CSV.read("cost.csv", DataFrame)
replace!(production_costs.Region, "Europe, Middle East, and Africa" => "EMEA");

In [10]:
production_costs

Row,Region,yr_2023_shoes,yr_2022_shoes,yr_2021_shoes,yr_2020_shoes,yr_2019_shoes,yr_2018_shoes,yr_2017_shoes,yr_2016_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,81.465,38.903,45.4594,38.9553,47.7326,42.1257,45.144,42.5316
2,EMEA,76.8032,40.0918,46.9997,40.8797,44.1989,37.9323,38.5832,43.1429
3,N ASIA,70.5533,37.6785,49.5428,38.7469,46.6495,47.2958,42.2007,41.6894
4,S ASIA,82.0352,39.3142,47.2482,35.5651,42.4247,44.6127,35.3256,39.6973
5,SE ASIA,75.738,41.2462,48.1081,36.4332,44.048,47.8443,44.7181,40.4575


In [11]:
# Define sizes
num_suppliers = size(availability)[1] #i
num_consumer_regions = size(demand)[1] # j = c(i) 
num_time = size(availability)[2] - 1;  
num_producer_regions = size(availability)[1] # p(i)

5

### Set-up

In [12]:
# Sets
suppliers = 1:num_suppliers
consumer_regions = 1:num_consumer_regions
producer_regions = 1:num_producer_regions
years = 1:num_time
years_incl_zero = 0:num_time

0:8

In [13]:
# Parameters and input data
A = Matrix(availability[:, 2:end]) #5x8
D = Matrix(demand[:, 2:end]) #5x8
R = Array(rev_year) #8x1
H = Array(holding_costs[:, 2]) #8x1
T = Matrix(shipping_costs[:, 6:end]) #25x8
W = Matrix(shipping_mapping[:, 2:end]) #encodes the prodcuer/consumer region for T 5x5
C = Matrix(production_costs[:, 2:end]); #5x8

### Creating the model

In [14]:
# Define parameters (choose 1 value for now)
alpha = 1
cost_of_shoes = 116.5

116.5

In [15]:
function run_model(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0);
    @variable(model, S[suppliers, consumer_regions, years] >= 0);
    @variable(model, E[suppliers, years_incl_zero] >= 0);
    @variable(model, M[consumer_regions, years] >= 0);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model (generic function with 1 method)

In [16]:
model_simple, X_values_simple, S_values_simple, E_values_simple, M_values_simple = run_model(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0x7ad4ea80
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+05, 1e+08]
Presolve removed 50 rows and 10 columns
Presolve time: 0.00s
Presolved: 120 rows, 320 columns, 755 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1819580e+35   3.840000e+32   1.181958e+05      0s
     140    1.1610159e+11   0.000000e+00   0.000000e+00      0s

Solved in 140 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.161015926e+11

User-callback calls 184, time in user-callback 0.00 sec


In [17]:
net_profit_simple = objective_value(model_simple)
holding_cost_simple = sum(H[t]*sum(E_values_simple[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_simple)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_simple)

Net profit: 1.161015926242511e11
Holding cost: 4.52122901032e9


In [18]:
# Plot graphs

# Total costs over t
# Region

# Holding quantity over t

# Herfindalhs over t


In [19]:
# Plot graphs across alphas
# Ave Holding quantity over t
# Ave Herfindalhs over t


## Impose Integrality

In [61]:
function run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] == 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    #@constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}


    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer (generic function with 1 method)

In [62]:
model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 130 rows, 330 columns and 570 nonzeros
Model fingerprint: 0x5298a01b
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 50 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (3 binary)
Found heuristic solution: objective 6.506531e+09

Root relaxation: objective 6.668555e+10, 102 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [63]:
net_profit_integer = objective_value(model_integer)
holding_cost_integer = sum(H[t]*sum(E_values_integer[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_integer)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_integer)

Net profit: 6.66855546185404e10
Holding cost: 2.624971531e9


In [23]:
C

5×8 Matrix{Float64}:
 81.465   38.903   45.4594  38.9553  47.7326  42.1257  45.144   42.5316
 76.8032  40.0918  46.9997  40.8797  44.1989  37.9323  38.5832  43.1429
 70.5533  37.6785  49.5428  38.7469  46.6495  47.2958  42.2007  41.6894
 82.0352  39.3142  47.2482  35.5651  42.4247  44.6127  35.3256  39.6973
 75.738   41.2462  48.1081  36.4332  44.048   47.8443  44.7181  40.4575

#### Uncertainty in Availability
Define uncertainty based on random vector for reductions in availability (0-0.25)

In [24]:
random_seed = 123
reductions = rand(10)/4

10-element Vector{Float64}:
 0.12040935937291677
 0.027990134617322582
 0.07914856064111536
 0.06231914953078713
 0.15841598210753186
 0.14257047503283585
 0.18744405834299466
 0.12344600797836136
 0.18600271957755043
 0.18487421033912638

In [64]:
function run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] == 0,  Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))

    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t] - A[i,t] * maximum(reductions)); #add uncertainty
    #@constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers));
    
    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer_uncertainty (generic function with 1 method)

In [65]:
model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(1, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0x469af179
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 87 rows and 124 columns
Presolve time: 0.00s
Presolved: 83 rows, 206 columns, 524 nonzeros
Found heuristic solution: objective 6.616999e+09
Variable types: 0 continuous, 206 integer (2 binary)
Found heuristic solution: objective 6.617004e+09

Root relaxation: objective 6.359501e+10, 84 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective 

In [66]:
net_profit_uncertainty = objective_value(model_uncertainty)
holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_uncertainty)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_uncertainty)

Net profit: 6.359500860042916e10
Holding cost: 1.4769445493400002e9


## Compare Models

In [67]:
#Outputs
models = ["Simple", "Integer Solution", "Integer + Uncertainty in Availability"]
net_profit = [net_profit_simple, net_profit_integer, net_profit_uncertainty]
holding_cost = [holding_cost_simple, holding_cost_integer, holding_cost_uncertainty]
output = DataFrame(Model = models, NetProfit = net_profit, HoldingCost = holding_cost)

Row,Model,NetProfit,HoldingCost
,String,Float64,Float64
1,Simple,1.16102e11,4.52123e9
2,Integer Solution,6.66856e10,2.62497e9
3,Integer + Uncertainty in Availability,6.3595e10,1.47694e9


In [68]:
#No uncertainty - only difference is the change in intial inventory
function run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, X_values_pre, E_values_pre)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] == 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,5] == X_values_pre[i,5]);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,5] == E_values_pre[i, 5]);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    #@constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}


    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_after_shock (generic function with 1 method)

## Determine Best Parameters
Test various parameters on 2016-2020

In [69]:
#Punishment on unmet demand
alpha_values = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1];
length(alpha_values)

7

In [70]:
#severity of availability shock
delta_values = [0.1, 0.2, 0.3, 0.4, 0.5];
#(1/0.3)

In [71]:
#Intialize Dictionaries for Results - 2016-2020
X_val = Dict()
S_val = Dict()
E_val = Dict()
M_val = Dict()

Dict{Any, Any}()

In [72]:
#Intialize Dictionaries for Results - 2021-2023
X_val_after = Dict()
S_val_after = Dict()
E_val_after = Dict()
M_val_after = Dict()

Dict{Any, Any}()

In [73]:
#Intialize Dataframe of values
results_16_20 = DataFrame(Model_Type = "", Alpha = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)
results_21_23 = DataFrame(Model_Type = "", Alpha = 0.0, Uncertainty_Range = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)

Row,Model_Type,Alpha,Uncertainty_Range,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64,Float64
1,,0.0,0.0,0.0,0.0


In [74]:
for i in 1:length(alpha_values)
    alpha = alpha_values[i]
    #1. Run non-robust model
    model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);
    net_profit_integer = objective_value(model_integer)
    holding_cost_integer = sum(H[t]*sum(E_values_integer[z,t] for z in suppliers) for t in 1:5)
    #Add outputs to dictionary
    name = string(alpha) * "_non_robust"
    X_val[name] = X_values_integer
    S_val[name] = S_values_integer
    E_val[name] = E_values_integer
    M_val[name] = M_values_integer
    #Append to DataFrame
    push!(results_16_20, ["Non Robust", alpha, net_profit_integer, holding_cost_integer])
    #2. Run robust model
    model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, 1:5, 0:5, A, D, R, H, T, W, C);
    net_profit_uncertainty = objective_value(model_uncertainty)
    holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[z,t] for z in suppliers) for t in 1:5)
    #Add outputs to dictionary
    name = string(alpha) * "_robust"
    X_val[name] = X_values_uncertainty
    S_val[name] = S_values_uncertainty
    E_val[name] = E_values_uncertainty
    M_val[name] = M_values_uncertainty
    #Append to DataFrame
    push!(results_16_20, ["Robust", alpha, net_profit_uncertainty, holding_cost_uncertainty])
    #Evalute for various shocks in 2021-2023
    println("hi")
    for j in 1:length(delta_values)
        A_shock = A
        for k in 1:5
            for l in 6:8
                Random.seed!(k+l)
                A_shock[k,l] = round(A_shock[k,l] * (1 - rand()/(1/delta_values[j])))
            end
        end
        #Test Non-Robust Model
        model_after_nonrobust, X_values_after_nonrobust, S_values_after_nonrobust, E_values_after_nonrobust, M_values_after_nonrobust = run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions, 6:8, 5:8, A_shock, D, R, H, T, W, C, X_values_integer, E_values_integer);
        net_profit_after_nonrobust = objective_value(model_after_nonrobust)
        holding_cost_after_nonrobust = sum(H[t]*sum(E_values_after_nonrobust[z,t] for z in suppliers) for t in 1:3)
        push!(results_21_23, ["Non Robust", alpha, delta_values[j], net_profit_after_nonrobust, holding_cost_after_nonrobust])
        #Add outputs to dictionary
        name = string(alpha) * string(delta_values[j]) * "_non_robust"
        X_val_after[name] = X_values_after_nonrobust
        S_val_after[name] = S_values_after_nonrobust
        E_val_after[name] = E_values_after_nonrobust
        M_val_after[name] = M_values_after_nonrobust
        #Test Robust Model
        model_after_robust, X_values_after_robust, S_values_after_robust, E_values_after_robust, M_values_after_robust = run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions,6:8, 5:8, A_shock, D, R, H, T, W, C, X_values_uncertainty, E_values_uncertainty);
        net_profit_after_robust = objective_value(model_after_robust)
        holding_cost_after_robust = sum(H[t]*sum(E_values_after_robust[z,t] for z in suppliers) for t in 1:3)
        push!(results_21_23, ["Robust", alpha, delta_values[j], net_profit_after_robust, holding_cost_after_robust])
        #Add outputs to dictionary
        name = string(alpha) * string(delta_values[j]) * "_robust"
        X_val_after[name] = X_values_after_robust
        S_val_after[name] = S_values_after_robust
        E_val_after[name] = E_values_after_robust
        M_val_after[name] = M_values_after_robust
    end

end
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 130 rows, 330 columns and 570 nonzeros
Model fingerprint: 0x1117cf74
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 50 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (3 binary)
Found heuristic solution: objective 6.506531e+09

Root relaxation: objective 6.668555e+10, 102 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [58]:
results_21_23

Row,Model_Type,Alpha,Uncertainty_Range,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64,Float64
1,,0.0,0.0,0.0,0.0
2,Non Robust,0.0,0.1,2.93485e8,0.0
3,Robust,0.0,0.1,2.93485e8,0.0
4,Non Robust,0.0,0.2,2.83758e8,0.0
5,Robust,0.0,0.2,2.83758e8,0.0
6,Non Robust,0.0,0.3,2.69687e8,0.0
7,Robust,0.0,0.3,2.69687e8,0.0
8,Non Robust,0.0,0.4,2.51924e8,0.0
9,Robust,0.0,0.4,2.51924e8,0.0


Model 2016-2020 with original data

In [37]:
model_preshock, X_values_preshock, S_values_preshock, E_values_preshock, M_values_preshock = run_model_integer_uncertainty(0.5, 
    suppliers, consumer_regions, producer_regions, 1:5, 0:5, A[:, 1:5], D[:, 1:5], R[1:5], H[1:5], T[:, 1:5], W, C[:, 1:5]);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 135 rows, 210 columns and 660 nonzeros
Model fingerprint: 0x67eb9e0c
Variable types: 0 continuous, 210 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+05, 1e+08]
Found heuristic solution: objective -5.53735e+10
Presolve removed 65 rows and 45 columns
Presolve time: 0.00s
Presolved: 70 rows, 165 columns, 430 nonzeros
Variable types: 0 continuous, 165 integer (0 binary)
Found heuristic solution: objective -3.88642e+10

Root relaxation: objective 6.251262e+10, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [38]:
net_profit_preshock = objective_value(model_preshock)
holding_cost_preshock = sum(H[t]*sum(E_values_preshock[i,t] for i in suppliers) for t in 1:5)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_preshock)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_preshock)

Net profit: 6.251261783273032e10
Holding cost: 6.8077135249e8


Model 2021-2023 with Supply Chain Shock

In [39]:
function run_model_shock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0, Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,5] == X_values_preshock[i, 5]);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,5] == E_values_preshock[i, 5]);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= 2*(D[j,t]/avg_price * sum(price_list_abs_dif)) - D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    
    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_shock (generic function with 1 method)

In [40]:
model_shock, X_values_shock, S_values_shock, E_values_shock, M_values_shock = run_model_shock(0.5, 
    suppliers, consumer_regions, producer_regions, 6:8, 5:8, A_shock, D, R, H, T, W, C);

LoadError: UndefVarError: `A_shock` not defined

In [41]:
net_profit_shock = objective_value(model_shock)
holding_cost_shock = sum(H[t]*sum(E_values_shock[i,t] for i in suppliers) for t in 1:3)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_shock)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_shock)

LoadError: UndefVarError: `model_shock` not defined

In [42]:
net_profit_pre_post = net_profit_shock + net_profit_preshock
holding_cost_pre_post = holding_cost_shock + holding_cost_preshock
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_pre_post)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_pre_post)

LoadError: UndefVarError: `net_profit_shock` not defined

In [43]:
#Outputs
models = ["No Shock", "Mild Shock (~25% Reduction)"]
net_profit = [net_profit_uncertainty, net_profit_pre_post ]
holding_cost = [holding_cost_uncertainty, holding_cost_pre_post]
output = DataFrame(Model = models, NetProfit = net_profit, HoldingCost = holding_cost)

LoadError: UndefVarError: `net_profit_pre_post` not defined